In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os
print(os.getcwd())

/mnt/c/Users/deela/datastorm_final


In [6]:
data_df = pd.read_csv('./data/indexed.csv')

/mnt/c/Users/deela/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
data_df.head()

,Unnamed: 0,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,commencement_dt,next_due_dt,termination_dt,termination_reason,main_holder_gender,...,main_holder_occupation,policy_snapshot_as_on,run_date,client_code,agent_code,product_code,product_name,policy_code,policy_snapshot,unique
0,79686,226.0,12,M,INFORCE,2018/12/28,2019/01/28,NaN,NaN,M,...,FLOUR & GRAIN MILL W,20181231,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331,2018-12-31,20181231 C100000
1,375697,175.0,20,M,INFORCE,2018/12/29,2019/01/29,NaN,NaN,M,...,CHENA CULTIVATOR,20181231,20200701,C100001,AG106517,HE001,HEALTH,POL7504,2018-12-31,20181231 C100001
2,592003,226.0,15,Y,INFORCE,2018/12/28,2019/12/28,NaN,NaN,M,...,FLOUR & GRAIN MILL W,20181231,20200701,C100002,AG106079,HE001,HEALTH,POL4435,2018-12-31,20181231 C100002
3,123460,450.0,12,H,INFORCE,2018/12/31,2019/06/30,NaN,NaN,F,...,MINISTER,20181231,20200701,C100003,AG105289,IN001,INVESTMENT,POL3879,2018-12-31,20181231 C100003
4,170518,450.0,12,Y,INFORCE,2018/12/28,2019/12/28,NaN,NaN,F,...,MINISTER,20181231,20200701,C100004,AG100675,IN001,INVESTMENT,POL6707,2018-12-31,20181231 C100004


In [39]:
def cat_to_int(df, columns, enc={}):
    df = df.copy()
    if enc == {} or len(columns) > len(enc) :
        maps = enc
        for col in columns:
            if col not in maps:
                mapping = {k: i for i,k in enumerate(df.loc[:,col].unique())}
                maps[col] = mapping
            df[col] = df[col].map(maps[col])
        return df, maps
    else:
        maps = enc
        for col in columns:
            df[col] = df[col].map(maps[col])
        return df

In [40]:
srt_df = data_df.sort_values(by=['policy_snapshot']).loc[:, ["client_code","product_code"]]
enc_df,map = cat_to_int(srt_df,["client_code","product_code"])

In [41]:
enc_df.head()

,client_code,product_code
0,0,0
17749,1,1
17748,2,0
17747,3,1
17746,4,1


In [42]:
enc_df["cnt"] = 1
enc_df.head()

,client_code,product_code,cnt
0,0,0,1
17749,1,1,1
17748,2,0,1
17747,3,1,1
17746,4,1,1


In [43]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(enc_df)

In [44]:
data_dummy.head()

,client_code,product_code,cnt,purchase_dummy
0,0,0,1,1
17749,1,1,1,1
17748,2,0,1,1
17747,3,1,1,1
17746,4,1,1,1


In [45]:
df_matrix = pd.pivot_table(enc_df, values='cnt', index='client_code', columns='product_code')

In [46]:
df_matrix.head()

product_code,0,1,2,3,4
client_code,,,,,
0,1.0,NaN,NaN,NaN,NaN
1,NaN,1.0,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN
3,NaN,1.0,NaN,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN


In [48]:
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['client_code'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(0, 3)


,client_code,product_code,scaled_purchase_freq


In [49]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [54]:
train_data_norm, test_data_norm = split_data(enc_df)
#train_data_dummy, test_data_dummy = split_data(data_dummy)
#train_data_norm, test_data_norm = split_data(data_dummy)

In [63]:
user_id = 'client_code'
item_id = 'product_code'
users_to_recommend = list(enc_df[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [64]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [70]:
name = 'popularity'
target = 'cnt'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)


Preparing data set.

Data has 505137 observations with 29491 users and 5 items.

Data prepared in: 0.498692s

505137 observations to process; with 5 unique items.

recommendations finished on 1000/631422 queries. users per second: 649351

...

recommendations finished on 310000/631422 queries. users per second: 1.26626e+06

recommendations finished on 311000/631422 queries. users per second: 1.26834e+06

+-------------+--------------+-------+------+
| client_code | product_code | score | rank |
+-------------+--------------+-------+------+
|      0      |      1       |  1.0  |  1   |
|      0      |      2       |  1.0  |  2   |
|      0      |      3       |  1.0  |  3   |
|      0      |      4       |  1.0  |  4   |
|      1      |      2       |  1.0  |  1   |
|      1      |      0       |  1.0  |  2   |
|      1      |      3       |  1.0  |  3   |
|      1      |      4       |  1.0  |  4   |
|      2      |      1       |  1.0  |  1   |
|      2      |      2       |  1.0  |  2   |
|      2      |      3       |  1.0  |  3   |
|      2      |      4       |  1.0  |  4   |
|      3      |      2       |  1.0  |  1   |
|      3      |      0       |  1.0  |  2   |
|      3      |      3       |  1.0  |  3   |
|      3      |      4       |  1.0  |  4   |
|      4      |      2       |  1.0  |  1   |
|      4      |      0       |  1.0  |  2   |
|      4      |      3       |  1.

recommendations finished on 312000/631422 queries. users per second: 1.27057e+06

recommendations finished on 313000/631422 queries. users per second: 1.27265e+06

In [67]:
name = 'cosine' 
target = 'cnt' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 505137 observations with 29491 users and 5 items.

Data prepared in: 0.460143s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 384000/631422 queries. users per second: 593058

recommendations finished on 385000/631422 queries. users per second: 593990

recommendations finished on 386000/631422 queries. users per second: 594898

+-------------+--------------+----------------------+------+
| client_code | product_code |        score         | rank |
+-------------+--------------+----------------------+------+
|      0      |      1       | 0.13400894403457642  |  1   |
|      0      |      2       | 0.06055343151092529  |  2   |
|      0      |      3       | 0.040576398372650146 |  3   |
|      0      |      4       | 0.038836777210235596 |  4   |
|      1      |      0       | 0.13400894403457642  |  1   |
|      1      |      2       | 0.09737014770507812  |  2   |
|      1      |      4       | 0.07135558128356934  |  3   |
|      1      |      3       | 0.05922418832778931  |  4   |
|      2      |      1       | 0.13400894403457642  |  1   |
|      2      |      2       | 0.06055343151092529  |  2   |
|      2      |      3       | 0.040576398372650146 |  3   |
|      2      |      4       | 0.038836777210235596 |  4   |
|      3      |      0       | 0.13400894403457642  |  1   |
|      3      |      2  

recommendations finished on 387000/631422 queries. users per second: 595612

In [68]:
name = 'pearson'
target = 'cnt'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 505137 observations with 29491 users and 5 items.

Data prepared in: 0.348703s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 627000/631422 queries. users per second: 498391

recommendations finished on 628000/631422 queries. users per second: 498669

recommendations finished on 629000/631422 queries. users per second: 498837

recommendations finished on 630000/631422 queries. users per second: 499190

recommendations finished on 631000/631422 queries. users per second: 499466

In [71]:
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [72]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/28160 queries. users per second: 326371

recommendations finished on 2000/28160 queries. users per second: 580552

recommendations finished on 3000/28160 queries. users per second: 779423

recommendations finished on 4000/28160 queries. users per second: 945180

...

recommendations finished on 25000/28160 queries. users per second: 653424

recommendations finished on 26000/28160 queries. users per second: 652873

recommendations finished on 27000/28160 queries. users per second: 657734

recommendations finished on 28000/28160 queries. users per second: 653473


Precision and recall summary statistics by cutoff
+--------+------------------------+------------------------+
| cutoff |     mean_precision     |      mean_recall       |
+--------+------------------------+------------------------+
|   1    | 0.00031960227272727197 | 0.00014174952651515183 |
|   2    | 0.0003728693181818176  | 0.00023323356331168748 |
|   3    | 0.0002959280303030302  | 0.0003220119724025973  |
|   4    | 0.00022194602272727264 | 0.00032201197240259755 |
|   5    | 0.00020419034090909098 | 0.0003220119724025972  |
|   6    | 0.00020419034090909106 | 0.0003220119724025971  |
|   7    | 0.00020419034090909046 | 0.00032201197240259766 |
|   8    | 0.00020419034090909117 | 0.0003220119724025973  |
|   9    | 0.00020419034090909095 | 0.00032201197240259755 |
|   10   | 0.00020419034090909117 | 0.00032201197240259733 |
+--------+------------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+-------------+------+-------+


In [74]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(enc_df), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='cnt', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)


Preparing data set.

Data has 631422 observations with 29501 users and 5 items.

Data prepared in: 0.487105s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 285000/631422 queries. users per second: 1.16815e+06

recommendations finished on 286000/631422 queries. users per second: 1.16822e+06

recommendations finished on 287000/631422 queries. users per second: 1.16858e+06

recommendations finished on 288000/631422 queries. users per second: 1.16769e+06

recommendations finished on 289000/631422 queries. users per second: 1.16724e+06

In [76]:
recom.to_dataframe().to_csv('recomendation.csv',index=False)